# Лабораторная работа 5. Деревья решений

### Задание. Построить дерево решений.
Необходимо для датасета из лекции "Состоится ли игра?" построить дерево решений.

Напишите и протестируйте функции вычисления энтропии (`entropy`) и прироста информации (`information_gain`): 

Создайте датасет "Состоится ли игра?" (вручную или прочитайте из файла):

In [16]:
import graphviz
import numpy as np
import pandas as pd

Напишите функцию построения дерева решений с помощью рекурсивных вызовов подготовленных функций:

In [17]:
def entropy(probabilities):
    probabilities = np.array(probabilities)
    mask = np.isinf(np.log2(probabilities))
    probabilities_log = np.where(mask, 0, np.log2(probabilities))
    return -np.sum(probabilities * probabilities_log)

def information_gain(info_a, info_b):
    return info_a - info_b

def recursive_build_decision_tree(tree, df, target, parent_node=None, info_a=None, prev_value=None):
    columns = df.columns[df.columns != target]
    if info_a is None:
        info_a = entropy(df[target].value_counts(normalize=True))
    
    if len(df[target].unique()) == 1:
        node_name = str(len(df[target])) + " " + str(df[target].unique()[0])
        tree.node(node_name)
        if parent_node is not None:
            tree.edge(parent_node, node_name, prev_value)
        return

    gains = {}
    for column in columns:
        probability_weights = df[column].value_counts(normalize=True)
        info_b = 0
        
        for value in probability_weights.index:
            target_values_probabilities = df[target][df[column] == value].value_counts(normalize=True)
            info_b += probability_weights[value] * entropy(target_values_probabilities)
        
        gains[column] = information_gain(info_a, info_b)

    best_feature = max(gains, key=gains.get)
    tree.node(best_feature)
    
    if parent_node is not None:
        tree.edge(parent_node, best_feature, prev_value)
        
    for value in df[best_feature].unique():
        recursive_build_decision_tree(tree, df[df[best_feature] == value].drop(best_feature, axis=1), target, best_feature, info_a - gains[best_feature], value)
        
def draw_decision_tree(df, target_column_name):
    tree = graphviz.Digraph('decision_tree')
    recursive_build_decision_tree(tree, df, target_column_name)
    tree.view()

Постройте дерево решений для датасета "Состоится ли игра?":

Визуализируйте построенное дерево (в любом виде):

In [18]:
game_df = pd.read_csv("will_there_be_a_game.csv", sep=",", encoding="windows-1251")
draw_decision_tree(game_df, "Игра")

Сравните полученные результаты с результатами функции из scikit-learn:

In [19]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

game_df = pd.read_csv("will_there_be_a_game.csv", sep=",", encoding="windows-1251")
x = game_df.drop("Игра", axis=1)
x = x.apply(LabelEncoder().fit_transform)
x

,Наблюдение,Температура,Влажность,Ветер
0,2,0,0,1
1,2,0,0,0
2,1,0,0,1
3,0,1,0,1
4,0,2,1,1
5,0,2,1,0
6,1,2,1,0
7,2,1,0,1
8,2,2,1,1
9,0,1,1,1


In [20]:
y = game_df["Игра"]
dec_tree = DecisionTreeClassifier()
dec_tree.fit(x, y)
tree = tree.export_graphviz(dec_tree, out_file="tree.dot", feature_names=x.columns, class_names=["Да", "Нет"], filled=True)